# 분석4: 뉴스 감성 영향 분석

## 목적
뉴스 긍정/부정이 집값에 미친 영향 분석

## 분석 내용
- 감성지수 vs 실거래가 상관관계
- 긍정/부정 뉴스 증가 시점 전후 비교
- 시차 분석 (Lag Analysis)
- 구별 민감도 차이

## 핵심 질문
1. 긍정 뉴스가 많으면 집값이 오르는가?
2. 반응 속도는 얼마나 되는가?
3. 어느 구가 뉴스에 가장 민감한가?

## 1. 라이브러리 설치 및 임포트

In [ ]:
# 필요한 라이브러리 설치 (처음 실행 시 주석 해제)
!pip install pandas numpy matplotlib seaborn plotly scipy konlpy transformers torch

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
from scipy.stats import pearsonr, spearmanr
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
# plt.rcParams['font.family'] = 'AppleGothic'  # Mac
plt.rcParams['axes.unicode_minus'] = False

## 2. 데이터 로드

### 2-1. 뉴스 크롤링 데이터 로드
팀원이 수집한 뉴스 데이터를 로드합니다.

In [ ]:
# 뉴스 데이터 로드 (파일 경로 수정 필요)
# news_df = pd.read_csv('../data/news_data.csv')

# 샘플 데이터 생성 (실제 데이터로 교체 필요)
np.random.seed(42)
dates = pd.date_range(start='2023-01-01', end='2024-12-31', freq='D')

# 샘플 뉴스 데이터 생성
news_df = pd.DataFrame({
    'date': np.random.choice(dates, 1000),
    'title': ['부동산 관련 뉴스 ' + str(i) for i in range(1000)],
    'content': ['뉴스 본문 내용 ' + str(i) for i in range(1000)],
    'source': np.random.choice(['중앙일보', '조선일보', '한겨레', 'MBC', 'SBS'], 1000),
    'region': np.random.choice(['강남구', '서초구', '송파구', '마포구', '용산구', '성동구'], 1000)
})

news_df['date'] = pd.to_datetime(news_df['date'])
news_df = news_df.sort_values('date').reset_index(drop=True)
print(f"뉴스 데이터: {len(news_df)}건")
news_df.head()

### 2-2. 실거래가 데이터 로드
공공데이터 API에서 수집한 실거래가 데이터를 로드합니다.

In [ ]:
# 실거래가 데이터 로드 (파일 경로 수정 필요)
# price_df = pd.read_csv('../data/estate_price.csv')

# 샘플 실거래가 데이터 생성
regions = ['강남구', '서초구', '송파구', '마포구', '용산구', '성동구']
months = pd.date_range(start='2023-01-01', end='2024-12-31', freq='M')

price_data = []
base_prices = {'강남구': 25, '서초구': 22, '송파구': 18, '마포구': 15, '용산구': 20, '성동구': 14}

for region in regions:
    base = base_prices[region]
    for i, month in enumerate(months):
        # 트렌드 + 노이즈
        price = base + (i * 0.1) + np.random.normal(0, 0.5)
        price_data.append({
            'date': month,
            'region': region,
            'avg_price': price  # 억원 단위 (평당)
        })

price_df = pd.DataFrame(price_data)
price_df['date'] = pd.to_datetime(price_df['date'])
print(f"실거래가 데이터: {len(price_df)}건")
price_df.head()

## 3. 감성 분석 (Sentiment Analysis)

### 3-1. 방법 1: 키워드 기반 감성 분석 (간단한 방법)

In [ ]:
# 부동산 관련 감성 키워드 사전
positive_keywords = [
    '상승', '급등', '호재', '활황', '회복', '상승세', '강세', '인기', '호황',
    '투자', '수익', '개발', '재건축', '호가', '신고가', '프리미엄', '완판',
    '청약', '당첨', '분양', '매수', '입주', '교통호재', 'GTX', '개통',
    '학군', '인프라', '공원', '편의시설', '안정', '전망좋음'
]

negative_keywords = [
    '하락', '급락', '폭락', '침체', '불황', '하락세', '약세', '위기',
    '거래절벽', '미분양', '매물증가', '매도', '손실', '역전세', '깡통전세',
    '규제', '대출', '금리', '세금', '종부세', '양도세', '취득세',
    '투기', '버블', '과열', '조정', '경기침체', '불안', '우려'
]

def calculate_sentiment_score(text):
    """키워드 기반 감성 점수 계산 (-1 ~ 1)"""
    if pd.isna(text):
        return 0
    
    text = str(text)
    pos_count = sum(1 for word in positive_keywords if word in text)
    neg_count = sum(1 for word in negative_keywords if word in text)
    
    total = pos_count + neg_count
    if total == 0:
        return 0
    
    return (pos_count - neg_count) / total

# 감성 점수 계산
news_df['sentiment_score'] = news_df['title'].apply(calculate_sentiment_score) + \
                              news_df['content'].apply(calculate_sentiment_score) * 0.5

# 정규화
news_df['sentiment_score'] = news_df['sentiment_score'].clip(-1, 1)

# 감성 레이블 추가
news_df['sentiment_label'] = news_df['sentiment_score'].apply(
    lambda x: '긍정' if x > 0.1 else ('부정' if x < -0.1 else '중립')
)

print("감성 분포:")
print(news_df['sentiment_label'].value_counts())

### 3-2. 방법 2: KoBERT 기반 감성 분석 (딥러닝 방법)
더 정확한 감성 분석을 원할 경우 사용

In [ ]:
# KoBERT 감성 분석 (선택적 - GPU 권장)
# 실행 시간이 오래 걸릴 수 있음

USE_BERT = False  # True로 변경하면 BERT 사용

if USE_BERT:
    from transformers import pipeline
    
    # 한국어 감성 분석 모델 로드
    sentiment_pipeline = pipeline(
        "sentiment-analysis",
        model="snunlp/KR-FinBert-SC",  # 금융 특화 한국어 BERT
        tokenizer="snunlp/KR-FinBert-SC"
    )
    
    def bert_sentiment(text):
        if pd.isna(text) or len(str(text).strip()) == 0:
            return 0
        try:
            # 텍스트 길이 제한 (512 토큰)
            text = str(text)[:500]
            result = sentiment_pipeline(text)[0]
            # positive -> 1, negative -> -1
            if result['label'] == 'positive':
                return result['score']
            else:
                return -result['score']
        except:
            return 0
    
    # 배치 처리 (시간 절약)
    from tqdm import tqdm
    tqdm.pandas()
    news_df['bert_sentiment'] = news_df['title'].progress_apply(bert_sentiment)
    print("BERT 감성 분석 완료!")

## 4. 월별 감성지수 집계

In [ ]:
# 월별 집계
news_df['year_month'] = news_df['date'].dt.to_period('M')

# 전체 월별 감성지수
monthly_sentiment = news_df.groupby('year_month').agg({
    'sentiment_score': ['mean', 'std', 'count'],
    'sentiment_label': lambda x: (x == '긍정').sum() / len(x) * 100  # 긍정 비율
}).reset_index()

monthly_sentiment.columns = ['year_month', 'avg_sentiment', 'sentiment_std', 'news_count', 'positive_ratio']
monthly_sentiment['date'] = monthly_sentiment['year_month'].dt.to_timestamp()

# 구별 월별 감성지수
regional_sentiment = news_df.groupby(['year_month', 'region']).agg({
    'sentiment_score': 'mean',
    'sentiment_label': lambda x: (x == '긍정').sum() / len(x) * 100
}).reset_index()

regional_sentiment.columns = ['year_month', 'region', 'avg_sentiment', 'positive_ratio']
regional_sentiment['date'] = regional_sentiment['year_month'].dt.to_timestamp()

print("월별 감성지수:")
monthly_sentiment.head(10)

## 5. 감성지수 vs 실거래가 상관관계 분석

In [ ]:
# 데이터 병합
price_df['year_month'] = price_df['date'].dt.to_period('M')

# 전체 평균 가격
monthly_price = price_df.groupby('year_month')['avg_price'].mean().reset_index()
monthly_price['date'] = monthly_price['year_month'].dt.to_timestamp()

# 감성지수와 가격 병합
merged_df = pd.merge(
    monthly_sentiment[['date', 'avg_sentiment', 'positive_ratio']],
    monthly_price[['date', 'avg_price']],
    on='date',
    how='inner'
)

# 상관계수 계산
corr_sentiment_price, p_value = pearsonr(
    merged_df['avg_sentiment'].dropna(), 
    merged_df['avg_price'].dropna()
)

print(f"감성지수 vs 집값 상관계수: {corr_sentiment_price:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"통계적 유의성: {'유의함' if p_value < 0.05 else '유의하지 않음'}")

## 6. 시차 분석 (Lag Analysis)

뉴스 감성이 집값에 영향을 미치기까지 시간이 걸리는지 분석

In [ ]:
def calculate_lag_correlation(df, sentiment_col, price_col, max_lag=6):
    """시차별 상관계수 계산"""
    results = []
    
    for lag in range(-max_lag, max_lag + 1):
        if lag < 0:
            # 가격이 감성보다 먼저 (가격 -> 감성 영향)
            sentiment = df[sentiment_col].iloc[-lag:].values
            price = df[price_col].iloc[:lag].values
        elif lag > 0:
            # 감성이 가격보다 먼저 (감성 -> 가격 영향)
            sentiment = df[sentiment_col].iloc[:-lag].values
            price = df[price_col].iloc[lag:].values
        else:
            sentiment = df[sentiment_col].values
            price = df[price_col].values
        
        if len(sentiment) > 5 and len(price) > 5:
            corr, p_val = pearsonr(sentiment, price)
            results.append({
                'lag': lag,
                'correlation': corr,
                'p_value': p_val,
                'significant': p_val < 0.05
            })
    
    return pd.DataFrame(results)

# 시차 분석 실행
lag_results = calculate_lag_correlation(merged_df, 'avg_sentiment', 'avg_price', max_lag=6)

print("시차별 상관계수:")
print("(양수 lag = 감성이 가격에 선행, 음수 lag = 가격이 감성에 선행)")
print(lag_results.to_string(index=False))

# 최적 시차 찾기
best_lag = lag_results.loc[lag_results['correlation'].abs().idxmax()]
print(f"\n가장 강한 상관관계 시차: {best_lag['lag']}개월 (r={best_lag['correlation']:.4f})")

## 7. 구별 민감도 분석

In [ ]:
# 구별 상관계수 계산
regional_correlation = []

for region in regions:
    # 해당 구의 감성지수
    region_sentiment = regional_sentiment[regional_sentiment['region'] == region][['date', 'avg_sentiment']]
    
    # 해당 구의 가격
    region_price = price_df[price_df['region'] == region][['date', 'avg_price']]
    
    # 병합
    region_merged = pd.merge(region_sentiment, region_price, on='date', how='inner')
    
    if len(region_merged) > 5:
        corr, p_val = pearsonr(
            region_merged['avg_sentiment'].dropna(),
            region_merged['avg_price'].dropna()
        )
        
        regional_correlation.append({
            'region': region,
            'correlation': corr,
            'p_value': p_val,
            'sensitivity': abs(corr)  # 절대값으로 민감도 측정
        })

sensitivity_df = pd.DataFrame(regional_correlation)
sensitivity_df = sensitivity_df.sort_values('sensitivity', ascending=False)

print("구별 뉴스 민감도 (상관계수 절대값 기준):")
print(sensitivity_df.to_string(index=False))

## 8. 시각화

### 8-1. 이중축 차트 (감성지수 + 집값)

In [ ]:
# Plotly 이중축 차트
fig = make_subplots(specs=[[{"secondary_y": True}]])

# 감성지수 (왼쪽 y축)
fig.add_trace(
    go.Scatter(
        x=merged_df['date'],
        y=merged_df['avg_sentiment'],
        name="감성지수",
        line=dict(color='blue', width=2),
        hovertemplate='날짜: %{x}<br>감성지수: %{y:.3f}<extra></extra>'
    ),
    secondary_y=False
)

# 집값 (오른쪽 y축)
fig.add_trace(
    go.Scatter(
        x=merged_df['date'],
        y=merged_df['avg_price'],
        name="평균 집값 (억원/평)",
        line=dict(color='red', width=2),
        hovertemplate='날짜: %{x}<br>집값: %{y:.2f}억<extra></extra>'
    ),
    secondary_y=True
)

fig.update_layout(
    title={
        'text': f'뉴스 감성지수 vs 집값 추이<br><sup>상관계수: {corr_sentiment_price:.3f}</sup>',
        'x': 0.5
    },
    xaxis_title="날짜",
    hovermode='x unified',
    template='plotly_white',
    legend=dict(x=0.01, y=0.99)
)

fig.update_yaxes(title_text="감성지수", secondary_y=False)
fig.update_yaxes(title_text="집값 (억원/평)", secondary_y=True)

fig.show()

### 8-2. 긍정/부정 뉴스 비교

In [ ]:
# 월별 긍정/부정 뉴스 비율
monthly_sentiment_dist = news_df.groupby(['year_month', 'sentiment_label']).size().unstack(fill_value=0)
monthly_sentiment_dist = monthly_sentiment_dist.div(monthly_sentiment_dist.sum(axis=1), axis=0) * 100
monthly_sentiment_dist = monthly_sentiment_dist.reset_index()
monthly_sentiment_dist['date'] = monthly_sentiment_dist['year_month'].dt.to_timestamp()

# Stacked Bar Chart
fig = go.Figure()

colors = {'긍정': '#2E7D32', '중립': '#9E9E9E', '부정': '#C62828'}

for sentiment in ['긍정', '중립', '부정']:
    if sentiment in monthly_sentiment_dist.columns:
        fig.add_trace(
            go.Bar(
                x=monthly_sentiment_dist['date'],
                y=monthly_sentiment_dist[sentiment],
                name=sentiment,
                marker_color=colors[sentiment],
                hovertemplate=f'{sentiment}: %{{y:.1f}}%<extra></extra>'
            )
        )

fig.update_layout(
    title='월별 뉴스 감성 분포',
    xaxis_title='날짜',
    yaxis_title='비율 (%)',
    barmode='stack',
    template='plotly_white',
    hovermode='x unified'
)

fig.show()

### 8-3. 시차 상관분석 시각화

In [ ]:
# 시차별 상관계수 시각화
fig = go.Figure()

# Bar chart
colors = ['#2E7D32' if r > 0 else '#C62828' for r in lag_results['correlation']]
fig.add_trace(
    go.Bar(
        x=lag_results['lag'],
        y=lag_results['correlation'],
        marker_color=colors,
        text=[f'{r:.3f}' for r in lag_results['correlation']],
        textposition='outside',
        hovertemplate='시차: %{x}개월<br>상관계수: %{y:.4f}<extra></extra>'
    )
)

# 유의수준 선
fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.5)

fig.update_layout(
    title={
        'text': '시차별 상관계수 (감성지수 → 집값)<br><sup>양수 시차 = 감성이 집값에 선행</sup>',
        'x': 0.5
    },
    xaxis_title='시차 (개월)',
    yaxis_title='상관계수',
    template='plotly_white',
    xaxis=dict(tickmode='linear', tick0=-6, dtick=1)
)

fig.show()

### 8-4. 구별 민감도 히트맵

In [ ]:
# 구별 민감도 Bar Chart
fig = px.bar(
    sensitivity_df,
    x='region',
    y='sensitivity',
    color='correlation',
    color_continuous_scale='RdYlGn',
    title='구별 뉴스 민감도 (상관계수 절대값)',
    labels={'sensitivity': '민감도', 'region': '지역구', 'correlation': '상관계수'},
    text=[f'{s:.3f}' for s in sensitivity_df['sensitivity']]
)

fig.update_traces(textposition='outside')
fig.update_layout(
    template='plotly_white',
    xaxis={'categoryorder': 'total descending'}
)

fig.show()

### 8-5. 구별 감성지수 vs 집값 산점도

In [ ]:
# 구별 산점도 (Facet)
regional_merged = pd.merge(
    regional_sentiment[['date', 'region', 'avg_sentiment']],
    price_df[['date', 'region', 'avg_price']],
    on=['date', 'region'],
    how='inner'
)

fig = px.scatter(
    regional_merged,
    x='avg_sentiment',
    y='avg_price',
    color='region',
    facet_col='region',
    facet_col_wrap=3,
    trendline='ols',
    title='구별 감성지수 vs 집값 관계',
    labels={'avg_sentiment': '감성지수', 'avg_price': '집값 (억원/평)', 'region': '지역구'}
)

fig.update_layout(template='plotly_white', showlegend=False)
fig.show()

## 9. 분석 결과 요약

In [ ]:
print("="*60)
print("📊 뉴스 감성 영향 분석 결과 요약")
print("="*60)

print(f"\n1️⃣ 감성지수 vs 집값 상관관계")
print(f"   - 상관계수: {corr_sentiment_price:.4f}")
print(f"   - P-value: {p_value:.4f}")
if corr_sentiment_price > 0.3:
    print(f"   → 긍정적 상관관계: 긍정 뉴스가 많을수록 집값 상승 경향")
elif corr_sentiment_price < -0.3:
    print(f"   → 부정적 상관관계: 긍정 뉴스가 많을수록 집값 하락 경향")
else:
    print(f"   → 약한 상관관계: 뉴스 감성과 집값의 직접적 관계 미약")

print(f"\n2️⃣ 시차 분석 (반응 속도)")
print(f"   - 최적 시차: {int(best_lag['lag'])}개월")
print(f"   - 해당 시차 상관계수: {best_lag['correlation']:.4f}")
if best_lag['lag'] > 0:
    print(f"   → 뉴스 감성이 {int(best_lag['lag'])}개월 후 집값에 반영되는 경향")
elif best_lag['lag'] < 0:
    print(f"   → 집값 변동이 {int(abs(best_lag['lag']))}개월 후 뉴스 감성에 영향")

print(f"\n3️⃣ 구별 민감도 순위")
for i, row in sensitivity_df.head(3).iterrows():
    print(f"   {sensitivity_df.index.get_loc(i)+1}위: {row['region']} (민감도: {row['sensitivity']:.4f})")

print("\n" + "="*60)

## 10. 인터랙티브 대시보드 (통합)

In [ ]:
# 통합 대시보드
from plotly.subplots import make_subplots

fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        '감성지수 vs 집값 추이',
        '시차별 상관계수',
        '월별 감성 분포',
        '구별 민감도'
    ),
    specs=[
        [{"secondary_y": True}, {}],
        [{}, {}]
    ],
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

# 1. 이중축 차트 (감성지수 + 집값)
fig.add_trace(
    go.Scatter(x=merged_df['date'], y=merged_df['avg_sentiment'], name='감성지수', line=dict(color='blue')),
    row=1, col=1, secondary_y=False
)
fig.add_trace(
    go.Scatter(x=merged_df['date'], y=merged_df['avg_price'], name='집값', line=dict(color='red')),
    row=1, col=1, secondary_y=True
)

# 2. 시차 상관계수
lag_colors = ['#2E7D32' if r > 0 else '#C62828' for r in lag_results['correlation']]
fig.add_trace(
    go.Bar(x=lag_results['lag'], y=lag_results['correlation'], name='상관계수', marker_color=lag_colors, showlegend=False),
    row=1, col=2
)

# 3. 월별 감성 분포
for sentiment in ['긍정', '부정']:
    if sentiment in monthly_sentiment_dist.columns:
        color = '#2E7D32' if sentiment == '긍정' else '#C62828'
        fig.add_trace(
            go.Scatter(
                x=monthly_sentiment_dist['date'],
                y=monthly_sentiment_dist[sentiment],
                name=sentiment,
                fill='tozeroy',
                line=dict(color=color)
            ),
            row=2, col=1
        )

# 4. 구별 민감도
sensitivity_colors = ['#2E7D32' if c > 0 else '#C62828' for c in sensitivity_df['correlation']]
fig.add_trace(
    go.Bar(x=sensitivity_df['region'], y=sensitivity_df['sensitivity'], name='민감도', marker_color=sensitivity_colors, showlegend=False),
    row=2, col=2
)

fig.update_layout(
    height=700,
    title_text='뉴스 감성 영향 분석 대시보드',
    template='plotly_white',
    showlegend=True,
    legend=dict(x=1.05, y=1)
)

fig.show()

## 11. 결과 저장

In [ ]:
# 분석 결과 CSV 저장
output_path = '../data/analysis_results/'
import os
os.makedirs(output_path, exist_ok=True)

# 월별 감성지수 저장
monthly_sentiment.to_csv(f'{output_path}monthly_sentiment.csv', index=False, encoding='utf-8-sig')

# 구별 민감도 저장
sensitivity_df.to_csv(f'{output_path}regional_sensitivity.csv', index=False, encoding='utf-8-sig')

# 시차 분석 결과 저장
lag_results.to_csv(f'{output_path}lag_analysis.csv', index=False, encoding='utf-8-sig')

# 뉴스 감성 결과 저장
news_df.to_csv(f'{output_path}news_with_sentiment.csv', index=False, encoding='utf-8-sig')

print("✅ 분석 결과가 저장되었습니다!")
print(f"   저장 위치: {output_path}")